In [1]:
import numpy as np
from openai import AsyncOpenAI
import os
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain_openai import OpenAI
from getpass import getpass
import json
import traceback
import time
import asyncio
from dotenv import load_dotenv

load_dotenv()

False

In [2]:
# Check if OPENAI_API_KEY environment variable is set
if "OPENAI_API_KEY" not in os.environ:
    print("Enter your OpenAI API key:")
    os.environ["OPENAI_API_KEY"] = getpass()

# evaluator setup
# openai.api_key = os.environ.get("OPENAI_API_KEY")
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
# openai.api_base = os.environ.get("OPENAI_API_BASE")
evaluator_model = "gpt-4-1106-preview"

Enter your OpenAI API key:


In [3]:
# llm = OpenAI(api_key=OPENAI_API_KEY)
llm = AsyncOpenAI(api_key=OPENAI_API_KEY)
# llm = OpenAI()

In [4]:
# get question
question_ind = np.random.choice(60)
question_data = json.load(open("G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/evaluations/evaluation_ques.json", "r"))[question_ind]
question = question_data["question"]
print(question)

How would the body react if prescribed medication was suddenly discontinued?


In [5]:
# ## Prompt 
# QUERY_PROMPT_MODEL_PT = PromptTemplate(
#     input_variables=["question", "context"],
#     template = """You are an AI language model assistant. Your task is to generate answer
#     by taking information from the relevant context provided from a vector 
#     database. By considering multiple perspectives on the user question, your goal is to help
#     the user understand the concept of the question asked that is also relevant to the context provided. 
#     Provide these answers with proper type setting.

#     Original question: {question}
#     Context : {context}
#     """,
# )

In [5]:
# imports
import warnings

# Suppress TypedStorage deprecation warning
warnings.filterwarnings("ignore", category=UserWarning, message="TypedStorage is deprecated.*")
warnings.filterwarnings("ignore", category=UserWarning, message="LangChainDeprecationWarning: The function `run` was deprecated.*")


import chromadb
from chromadb.utils import embedding_functions

import json
import chromadb
from chromadb.config import Settings
from chromadb import EmbeddingFunction
from chromadb.utils import embedding_functions
from sentence_transformers import SentenceTransformer
from langchain_openai import OpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate


In [6]:
def get_context_with_max_similarity(user_question):
    # Initialize SentenceTransformer model
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L12-v2", normalize_embeddings=True)

    # Embed the user's question
    user_question_embedding = sentence_transformer_ef([user_question])[0]

    # Perform the query using Chroma
    search_results = collection_2013.query(query_embeddings=[user_question_embedding], n_results=5)

    # Find the index of the context with the maximum similarity score
    max_similarity_index = search_results['distances'][0].index(max(search_results['distances'][0]))

    # Get the context with the maximum similarity score
    context_with_max_similarity = search_results['documents'][0][max_similarity_index]

    return context_with_max_similarity

chroma_internet_client = chromadb.HttpClient(host='16.171.68.145', port=8000, settings=Settings(allow_reset=True))

collection_2013 = chroma_internet_client.get_collection('2013pubmed')

In [6]:
QUERY_PROMPT_MODEL = f"""
You are an AI language model assistant. Your task is to generate answer
    by taking information from the relevant context provided from a vector 
    database. By considering multiple perspectives on the user question, your goal is to help
    the user understand the concept of the question asked that is also relevant to the context provided. 
    Provide these answers with proper type setting.

    Original question: {question}
    Context : {context}
"""
#{context}



In [7]:
for i in range(10):
    try:
        chat_completion = await llm.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[{"role": "user", "content": QUERY_PROMPT_MODEL}],
            stream=False,
        )
        if isinstance(chat_completion, tuple):
            # Handle tuple response
            print("Received tuple response:", chat_completion)
            # Optionally, retry or handle the tuple response as needed
            continue

        if isinstance(chat_completion, dict):
            # Not stream
            answer = chat_completion.choices[0].message.content
        else:
            # Stream
            # for token in chat_completion:
            #     print(type(token))
            #     print(token)
            #     print(i)
            # Access the answer outside the loop
            answer = chat_completion.choices[0].message.content
        break
    except Exception as exc:
        print(traceback.format_exc())
        print(exc)
        if i != 9:
            print(f"Retrying... (i = {i})")
            time.sleep(3)

print(answer)


Consistently inadequate sleep can have various potential long-term effects on both physical and mental health. Some of the consequences include:

1. **Cognitive Impairment:** Chronic sleep deprivation can impair cognitive function, leading to difficulties in concentration, memory, and decision-making.

2. **Mood disorders:** Lack of sleep is closely linked to mood disorders such as depression and anxiety. Prolonged sleep deprivation can exacerbate these conditions.

3. **Weakened immune system:** Inadequate sleep weakens the immune system, making individuals more susceptible to infections and illnesses.

4. **Weight gain and obesity:** Sleep deprivation can disrupt hormones that regulate appetite, leading to increased food cravings and weight gain over time.

5. **Increased risk of chronic diseases:** Long-term inadequate sleep has been associated with a higher risk of developing chronic conditions such as diabetes, heart disease, and hypertension.

6. **Impaired physical performance:*

In [8]:
eval_prompt = f"""
        Your task is to evaluate a student's response to a given exercise. In the exercise, the student is provided with some general context consisting of 3 titles and abstracts of medical articles.
        The student is furthermore asked a question, which he should answer correctly making use of the provided context.
        The exercise tests the student's abilities regarding grammar, reading comprehension and logical reasoning. The student's answer starts after the *** symbol.
        Please provide your general assessment about the answer provided by the student (the part after the *** symbol).
        Is it correct? Is it grammatically correct? Is it consistent with the given context?
        Furthermore, grade the student’s answer in terms of grammar, coherence, consistency with the context and whether it is correct or not. Moreover, please provide your best guess of what the academic degree of the student might be, as reflected from the answer. Choose from possible 4 possible categories: A: no degree. B: bachelor's degree. C: master's degree. D: doctoral degree. Use the following grade format: Grammar: #/10, Coherence: #/10, Context: #/10, Correctness: #/10, where the "#" should be replaces by a number between 0 (worst) and 10 (best).
        Context: {context}
        Question: {question}
        Answer: *** {answer}
"""

In [9]:
for i in range(10):
    try:
        chat_completion = await llm.chat.completions.create(
            model=evaluator_model,
            messages=[{"role": "user", "content": QUERY_PROMPT_MODEL}],
            stream=False,
        )
        if isinstance(chat_completion, tuple):
            # Handle tuple response
            print("Received tuple response:", chat_completion)
            # Optionally, retry or handle the tuple response as needed
            continue

        if isinstance(chat_completion, dict):
            # Not stream
            answer = chat_completion.choices[0].message.content
        else:
            # Stream
            # for token in chat_completion:
            #     print(type(token))
            #     print(token)
            #     print(i)
            # Access the answer outside the loop
            answer = chat_completion.choices[0].message.content
        break
    except Exception as exc:
        print(traceback.format_exc())
        print(exc)
        if i != 9:
            print(f"Retrying... (i = {i})")
            time.sleep(3)

print(answer)


It seems there's no context provided in your message. However, I can provide general information about the potential long-term effects of consistently inadequate sleep based on existing scientific knowledge. 

**Potential Long-Term Effects of Inadequate Sleep:**

1. **Cognitive Impairment**: Chronic sleep deprivation can lead to difficulties in concentration, memory, and cognitive processing. This impairment can affect job performance, academic learning, and daily activities.

2. **Mood Disorders**: Inadequate sleep has been linked to mental health issues such as depression, anxiety, and irritability. Over time, lack of sleep can aggravate these symptoms and potentially lead to more severe mental illnesses.

3. **Cardiovascular Health Risks**: Long-term sleep deprivation has been associated with increased risk of heart disease, hypertension, stroke, and coronary artery disease due to increased stress hormone levels and blood pressure.

4. **Weight Gain and Obesity**: Poor sleep pattern

In [ ]:
llm_chain = LLMChain(prompt=QUERY_PROMPT, llm=llm)

###### Chroma db query